In [2]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.models import load_model
import joblib
import tensorflow as tf
from flask import Flask, request, jsonify

In [4]:

app = Flask(__name__)

# Load assets once at startup
model = load_model("model_recommender.keras")
tfidf = pickle.load(open("tfidf_vectorizer.pkl", "rb"))
course_embeddings = np.load("course_embeddings.npy")
courses = pd.read_csv("udemy_courses (1).csv")
encoder_model = tf.keras.models.load_model('course_encoder_model.keras')

# === Define recommendation function ===
def recommend_related(course_id, courses_df, embeddings, top_n=10):
    print(type(course_id))
    if course_id not in courses_df['id'].values:
        raise ValueError(f"Course ID {course_id} not found in dataset.")

    idx = courses_df[courses_df['id'] == course_id].index[0]
    course_vec = embeddings[idx].reshape(1, -1)

    similarities = cosine_similarity(course_vec, embeddings).flatten()
    courses_df['similarity'] = similarities

    recommendations = courses_df[courses_df['id'] != course_id] \
        .sort_values(by='similarity', ascending=False) \
        .head(top_n)

    return recommendations['id'].tolist() 
    # return recommendations[['id', 'title', 'similarity']]
## cf based  autoencoder

# Recommendation function (CF-auto encoder)
def recommend_courses(user_id, interaction_matrix_path='./CF-Autoencoder/interaction_matrix.npy',
                      course_data_path='udemy_courses (1).csv',
                      model_path='./CF-Autoencoder/autoencoder_model.keras',
                      user_encoder_path='./CF-Autoencoder/user_encoder.pkl',
                      course_encoder_path='./CF-Autoencoder/course_encoder.pkl',
                      top_n=5):
    try:
        autoencoder = load_model(model_path)
        user_encoder = joblib.load(user_encoder_path)
        course_encoder = joblib.load(course_encoder_path)
        courses_df = pd.read_csv(course_data_path)
        interaction_matrix = np.load(interaction_matrix_path)
    except Exception as e:
        return {"error": f"Error loading model or encoders: {e}"}

    if user_id not in user_encoder.classes_:
        return {"error": "User not found in encoder."}

    interaction_matrix = (interaction_matrix > 0).astype(int)
    user_idx = user_encoder.transform([user_id])[0]

    try:
        user_input = interaction_matrix[user_idx].reshape(1, -1)
        preds = autoencoder.predict(user_input, verbose=0)[0]
    except Exception as e:
        return {"error": f"Error during prediction: {e}"}

    already_taken = set(np.where(interaction_matrix[user_idx] > 0)[0])
    top_indices = np.argsort(preds)[::-1]

    recommended_indices = [idx for idx in top_indices if idx not in already_taken][:top_n]

    if not recommended_indices:
        return {"warning": "No new recommendations found."}

    try:
        recommended_course_ids = course_encoder.inverse_transform(recommended_indices)
    except Exception as e:
        return {"error": f"Error decoding course IDs: {e}"}

    return recommended_course_ids.tolist()

@app.route('/recommend-cf', methods=['POST'])
def recommend_cf():
    user_id = request.json.get('user_id')
    
    if not user_id:
        return jsonify({"error": "Missing user_id parameter"}), 400
    print(type(user_id))
    user_id=int(user_id)
    result = recommend_courses(user_id)
    return jsonify(result)


# === Flask route that returns JSON ===
@app.route('/recommend-by-id', methods=['POST'])
def recommend_by_id():
    data = request.get_json()
    course_id = data.get("course_id")
    course_id = int(course_id)
    if course_id is None:
        return jsonify({"error": "Missing course_id"}), 400

    try:
        recs = recommend_related(course_id, courses, course_embeddings)
        return jsonify(recs)  # Return the list directly as JSON
    except ValueError as e:
        return jsonify({"error": str(e)}), 404

@app.route('/recommend', methods=['POST'])
def recommend():
    data = request.get_json()
    query = data.get("query")

    if not query:
        return jsonify({"error": "Missing query"}), 400

    query_vec = tfidf.transform([query]).toarray()
    query_emb = model.predict(query_vec)

    similarities = cosine_similarity(query_emb, course_embeddings).flatten()
    top_indices = similarities.argsort()[::-1][:5]

    recommended_course_ids = courses.iloc[top_indices]['id'].tolist()
    return jsonify(recommended_course_ids)

if __name__ == '__main__':
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step


127.0.0.1 - - [08/May/2025 17:32:46] "POST /recommend HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


127.0.0.1 - - [08/May/2025 17:32:58] "POST /recommend HTTP/1.1" 200 -


### return ids 
